# Gene Regulatory Network 2
***
Here we illustrate how to implement the model describet in [Sturrock M, Hellander A, Matzavinos A, Chaplain MAJ. 2013 Spatial stochastic modelling of the Hes1 gene regulatory network: intrinsic noise can explain heterogeneity in embryonic stem cell differentiation. J R Soc Interface 10:20120988]

This model uses the cell_coarse mesh (slide 65)
***
## Setup the Environment
***

In [ ]:
import spatialpy

***
## Create the Gene Regulatory Network 2 Model
***

In [ ]:
def create_generegulatorynetwork2(parameter_values=None):
    model = spatialpy.Model(name='Gene-Regulatory-Network-2')

    # Define Domain Type IDs as constants of the Model
    model.NUCLEUS = 'Nucleus'
    model.CYTOPLASM = 'Cytoplasm'

    # Domain
    domain = spatialpy.Domain(
        0, (-2.0671e-06, 2.0671e-06), (-2.0671e-06, 2.0671e-06),
        (-2.0671e-06, 2.0671e-06), rho0=1, c0=10, P0=100, gravity=None
    )

    import os
    home = os.path.expanduser('~')
    # Domain Lattices
    ipa_lattice1 = spatialpy.XMLMeshLattice(
        os.path.join(home, 'V2-Spatial-Examples/Gene-Regulatory-Network.proj/Domain-Files/GRN_Spatial.mesh.xml'),
        subdomain_file=os.path.join(home, 'V2-Spatial-Examples/Gene-Regulatory-Network.proj/Domain-Files/GRN_Spatial.subdomains.txt')
    )

    # Domain Actions
    domain.add_fill_action(
        lattice=ipa_lattice1, enable=True, apply_action=True,
        mass=1, vol=1, rho=1, nu=0, c=10, fixed=False
    )

    model.add_domain(domain, allow_all_types=True)

    model.staticDomain = True

    # Variables
    G_F = spatialpy.Species(name='G_F', diffusion_coefficient=0, restrict_to=[model.NUCLEUS, model.CYTOPLASM])
    G_0 = spatialpy.Species(name='G_0', diffusion_coefficient=0, restrict_to=[model.NUCLEUS, model.CYTOPLASM])
    P = spatialpy.Species(name='P', diffusion_coefficient=1e-12, restrict_to=[model.NUCLEUS, model.CYTOPLASM])
    mRNA = spatialpy.Species(name='mRNA', diffusion_coefficient=1e-12, restrict_to=[model.NUCLEUS, model.CYTOPLASM])
    model.add_species([
        G_F, G_0, P, mRNA
    ])

    # Initial Conditions
    G_F_ic = spatialpy.PlaceInitialCondition(species='G_F', count=1, location=[0, 0, 0])
    model.add_initial_condition([
        G_F_ic
    ])

    # Parameters
    mu = spatialpy.Parameter(name='mu', expression='0.002')
    kappa = spatialpy.Parameter(name='kappa', expression='0.015')
    ka = spatialpy.Parameter(name='ka', expression='1.6605778811e-20')
    kd = spatialpy.Parameter(name='kd', expression='0.01')
    gamma_m = spatialpy.Parameter(name='gamma_m', expression='0.0006')
    gamma_p = spatialpy.Parameter(name='gamma_p', expression='0.0003')
    model.add_parameter([
        mu, kappa, ka, kd, gamma_m, gamma_p
    ])

    # Reactions
    R1 = spatialpy.Reaction(
        name='R1', rate='ka', restrict_to=[model.NUCLEUS, model.CYTOPLASM],
        reactants={'G_F': 1, 'P': 1}, products={'G_0': 1}
    )
    R2 = spatialpy.Reaction(
        name='R2', rate='kappa', restrict_to=[model.CYTOPLASM],
        reactants={'mRNA': 1}, products={'mRNA': 1, 'P': 1}
    )
    R3 = spatialpy.Reaction(
        name='R3', rate='mu', restrict_to=[model.NUCLEUS, model.CYTOPLASM],
        reactants={'G_F': 1}, products={'G_F': 1, 'mRNA': 1}
    )
    R4 = spatialpy.Reaction(
        name='R4', rate='gamma_m', restrict_to=[model.NUCLEUS, model.CYTOPLASM],
        reactants={'mRNA': 1}, products={}
    )
    R5 = spatialpy.Reaction(
        name='R5', rate='gamma_p', restrict_to=[model.NUCLEUS, model.CYTOPLASM],
        reactants={'P': 1}, products={}
    )
    R6 = spatialpy.Reaction(
        name='R6', rate='kd', restrict_to=[model.NUCLEUS, model.CYTOPLASM],
        reactants={'G_0': 1}, products={'G_F': 1, 'P': 1}
    )
    model.add_reaction([
        R1, R2, R3, R4, R5, R6
    ])

    # Timespan
    tspan = spatialpy.TimeSpan.arange(43.221610805402705, t=86400, timestep_size=1)
    model.timespan(tspan)
    return model

### Instantiate the model

In [ ]:
model = create_generegulatorynetwork2()

In [ ]:
model.domain

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    kwargs = {
        # 'number_of_trajectories': 1,
        # 'seed': None,
    }
    return kwargs

***
## Run the Simulation
***

In [ ]:
kwargs = configure_simulation()
results = model.run(**kwargs)

***
## Visualization
***

In [ ]:
results.plot_species('G_F', animated=True, width='auto', height='auto')